# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [ ]:
import pandas as pd


df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv")

df.head()

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [ ]:
df.columns = df.columns.str.lower().str.replace(" ", "_")
df

In [ ]:
# Filter customers who have total_claim_amount greater than 1000 and response is "Yes"
filtered_df = df[(df['total_claim_amount'] > 1000) & (df['response'] == 'Yes')]

# Display the new DataFrame
print(filtered_df)


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [ ]:
# Filter the DataFrame for customers who responded "Yes" to the marketing campaign
df_yes_response = df[df['response'] == 'Yes']

# Create a pivot table to calculate the average total_claim_amount by policy_type and gender
avg_claim_by_policy_gender = df_yes_response.pivot_table(
    values='total_claim_amount',  # The column to compute average for
    index='policy_type',           # Group by policy type
    columns='gender',              # Group by gender
    aggfunc='mean',                # Calculate the average
)

# Round the values to 2 decimal places for better readability
avg_claim_by_policy_gender = avg_claim_by_policy_gender.round(2)

# Display the result
print(avg_claim_by_policy_gender)


3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [ ]:
# Count the number of customers per state
state_customer_count = df['state'].value_counts()

# Filter the states where the number of customers is greater than 500
state_customer_count_filtered = state_customer_count[state_customer_count > 500]

# Display the result
print(state_customer_count_filtered)


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [ ]:
# Group the data by education level and gender, and calculate the max, min, and median of customer_lifetime_value
lifetime_value_stats = df.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['max', 'min', 'median'])

# Display the result
print(lifetime_value_stats)


## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here